# Exploratory data analysis (EDA)

Exploratory data analysis, or **EDA**, means just that. We are going look at and summarize our idea. The data cleaning that we've already done, the looking for missings and understanding our variables, is actually part of EDA. In these notes, howerver, we'll focus on **grouping**, or **aggregating**, and summarizing our data.

[DataCamp](https://www.datacamp.com/community/tutorials/exploratory-data-analysis-python), of course, has a tutorial on some EDA ideas, perhaps more related to what people are doing in machine learning.

[Chapter 10](https://wesmckinney.com/book/data-aggregation.html) of *Python for Data Analysis* discusses aggregation and summarizing our data. [Exploratory Data Analysis](https://aeturrell.github.io/coding-for-economists/data-exploratory-analysis.html) from *Coding for Economists* has data aggregation and summary examples.




## pandas profiling

We can install `pandas-profiling` via Anaconda. To use this package, you'll need to type the following in the terminal (Mac) or command prompt (Windows). You'll need to restart VS Code.

```
conda install -c conda-forge pandas-profiling
```

You can [read all about it on their Github page](https://github.com/pandas-profiling/pandas-profiling). It's ridiculous what it can do. 

Let's do our usual set-up, but bring `ProfileReport` from `pandas_profiling`.

In [ ]:
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport

uw = pd.read_csv('https://github.com/aaiken1/fin-data-analysis-python/raw/main/data/zestimatesAndCutoffs_byGeo_uw_2017-10-10_forDataPage.csv')


This will create the report.

In [ ]:
#profile = ProfileReport(uw, title="Pandas Profiling Report for UW Zillow Data")

We can now view the report in a Jupyter widget. It will take a couple of minutes to summarize everything. 

In [ ]:
#profile.to_widgets()

You can also create an HTML file. Using the code below, the file will end up in the same folder as your `.ipynb` notebook. I've commented it out so that it doesn't run and create large file everytime.

In [ ]:
#profile.to_file("uw_profile.html")